In [179]:
#МАШИНЫ

import pandas as pd
import random
import math
import time

start_time = time.time()

buyers = pd.DataFrame()

data = pd.read_csv("Car_sales.csv")

data['параметр 0'] = data['Price_in_thousands']
data['параметр 1'] = data['Horsepower']
data['параметр 2'] = data['Fuel_efficiency']

buyers.insert(0, "параметр 0", round(data['параметр 0']))
buyers.insert(1, "параметр 1", round(data['параметр 1'] * 0.1))
buyers.insert(2, "параметр 2", round(data['параметр 2'] * 0.1))

buyers = buyers.dropna()
buyers = buyers.reset_index(drop=True)

Qual_Amount = buyers.shape[1] - 1
print(Qual_Amount)
# buyers = data
print(buyers)
generation = buyers.copy()
Counter = 3

2
     параметр 0  параметр 1  параметр 2
0          22.0        14.0         3.0
1          28.0        22.0         2.0
2          42.0        21.0         2.0
3          24.0        15.0         3.0
4          34.0        20.0         2.0
..          ...         ...         ...
148        24.0        16.0         2.0
149        28.0        17.0         2.0
150        29.0        17.0         2.0
151        46.0        24.0         2.0
152        36.0        20.0         2.0

[153 rows x 3 columns]


In [180]:
#Функция снижения температуры
def DecreaseTemperature(initialTemperature, i):
    T = initialTemperature * 0.1 / i
    return T

#Вычисление вероятности перехода в худшее состояние
def GetTransitionProbability(current, state, T):
    if (state <= current):
        P = 1
    else:
        P = math.exp(-abs(state - current)/T)
    return P

def IsTransition(probability):
    value = random.randint(1, 99)/100
    if (value <= probability):
        a = 1
    else:
        a = 0
    return a

In [181]:
#Генерация состояния-кандидата
def GenerateCandidate(fr_name, energy):
    candidate = []
    counter = 0
    for i in range(0, Qual_Amount + 1):
        name1 = 'параметр ' + str(i)
        Quality = random.randint(int(min(fr_name[name1][:])), int(max(fr_name[name1][:])))
        candidate.append(Quality)
    return candidate

In [182]:
#Подсчёт количества покупателей при данном векторе товара
def Amount_of_Buyers(fr_name, vector):
    n = fr_name.shape[0]
    counter = 0
    for j in range (0, n):
        condition = (vector[0] <= fr_name['параметр 0'][j])
        for i in range(1, Qual_Amount + 1):
            name = 'параметр ' + str(i)
            condition &= vector[i] >= fr_name[name][j]
        if condition:
            counter += 1
    return counter

#Подсчёт прибыли
def CalculateEnergy(fr_name, vector):
    profit = vector[0]
    for i in range(1, Qual_Amount + 1):
        profit -= vector[i]
    profit = profit * Amount_of_Buyers(fr_name, vector)
    return profit

In [183]:
#Реализация алгоритма имиатции отжига
def SimulatedAnnealing(fr_name, initialTemperature, endTemperature):
    state = fr_name.loc[0][:]
    
    currentEnergy = CalculateEnergy(fr_name, state)
    
    T = initialTemperature
    
    for i in range (1, 100):   
        stateCandidate = GenerateCandidate(fr_name, currentEnergy)
        candidateEnergy = CalculateEnergy(fr_name, stateCandidate)
        
        if(candidateEnergy > currentEnergy): 
            currentEnergy = candidateEnergy
            state = stateCandidate
        else:
            p = GetTransitionProbability(candidateEnergy, currentEnergy, T)
            if (IsTransition(p) == 1):
                currentEnergy = candidateEnergy
                state = stateCandidate       
        
        T = DecreaseTemperature(initialTemperature, i)
       
        
        if(T <= endTemperature):
            break
    print(state)
    print(currentEnergy)
    return currentEnergy

SimulatedAnnealing(buyers, 13000, 0.1)
print("--- %s seconds ---" % (time.time() - start_time))

[67, 33, 4]
150
--- 2.52937912940979 seconds ---
